In [34]:
import  config
import re
from nltk.tokenize import wordpunct_tokenize
from scipy.spatial.distance import cdist, cosine

from nltk.stem.snowball import RussianStemmer
from nltk.tokenize import wordpunct_tokenize
import numpy as np

from plyvel import DB


good_sym = re.compile(u'^[А-Яа-яЁёA-Za-z]+$')

stemmer = RussianStemmer()
w2v_model = None


def load_w2v():
    global w2v_model
    print 'loading w2v'
    w2v_model = DB(config.w2v_path)
    print 'loaded'


def get_vector(word):
    try:
        return np.fromstring(w2v_model.get(word.encode('utf-8')), np.float16)
    except:
        return None


In [49]:
load_w2v()

loading w2v
loaded


In [4]:
text1 = u"""
Ресторан восточной кухни находится в самом центре Казани. Посетителей ждут восхитительные блюда азербайджанской, татарской и узбекской кухни от лучших шеф-поваров Казани. Этот шикарный ресторан идеально подходит для любого повода: будь то свадьба, юбилей, выпускной, личный праздник. В «Султанате» несколько залов: банкетный, каминный, ресторан, VIP-зал, караоке, детская игровая комната. В банкетном зале можно провести любой закрытый праздник: от деловой презентации до дня рождения, есть возможность пригласить любимых артистов. Каминный зал — это отдельный уютный зал с шикарной обстановкой и камином, который идеально подходит для проведения торжеств и свадеб. В ресторане гостей угощают лучшими блюдами европейской и восточной кухни. Кроме этого, в «Султанате» есть специальный зал караоке, где интерьер подобран для вечернего отдыха и желающих раскрыть душу в любимых песнях. Для посетителей с детьми работает детская комната. Гостями ресторана были такие люди, как Стивен Сигал, Александр Иванов и группа «Рондо», Салават Фатхетдинов и другие.
"""

In [22]:
text2 = u"""
Этот бар действительно необычен. Его создавали специально для любителей рока. Например, оформление каждого столика уникально и посвящено какому-то одному рок-исполнителю или группе. А счет здесь приносят в мышеловке: нет-нет, она не опасна! Это просто держатель для бумаг: по задумке авторов такое исполнение должно напомнить посетителям о том, где раздают бесплатный сыр. Что касается музыки, то здесь ее очень много. Плейлист, в котором почти шесть тысяч композиций, подбирал лично владелец заведения. А еще бар гордится своим звуком: говорят, такого нет не то, что в Казани, – даже в Москве редко где встретишь. Иногда тут проходят концерты. Чтобы послушать некоторые коллективы, придется заплатить за вход. Но в основном все мероприятия бесплатны. А кино тут показывают особенное: оно так или иначе связано с рок-н-роллом. Это может быть и художественный фильм, и документальный. Картины на большом экране демонстрируются не каждый день, расписание можно узнать заранее. За просмотр плата не взимается. Кроме всего прочего здесь можно вкусно поужинать: все блюда из меню делаются только на месте. Даже соус барбекю шеф-повар не покупает, а готовит сам. И говорят, что когда на ольховых щепках коптятся яблоки, аромат из кухни идет умопомрачительный!
"""

In [51]:
text12 = [t.lower() for t in wordpunct_tokenize(text1) if good_sym.match(t)]
text22 = [t.lower() for t in wordpunct_tokenize(text2) if good_sym.match(t)]
print len(text12), len(text22)

143 189


In [52]:
vecs1 = [get_vector(t) for t in text12]
vecs2 = [get_vector(t) for t in text22]

In [50]:
query =u'рок'
qv = get_vector(query)


[  6.62841797e-02   1.60522461e-02   4.76074219e-02  -2.13745117e-01
   1.62719727e-01   4.21386719e-01   7.08618164e-02   2.63427734e-01
   4.49951172e-01   2.57080078e-01   7.45605469e-01   3.76220703e-01
   3.46191406e-01  -2.27416992e-01   5.07812500e-01  -4.88586426e-02
   4.33105469e-01   1.51000977e-01  -8.83178711e-02   2.98583984e-01
   1.11389160e-01  -4.43420410e-02   2.63916016e-01  -2.06787109e-01
   2.53417969e-01  -3.18603516e-01   3.95996094e-01  -5.37414551e-02
  -8.69750977e-02   8.72192383e-02  -1.96166992e-01  -1.82739258e-01
   3.93798828e-01   3.29833984e-01  -8.77075195e-02   2.84423828e-01
   2.01782227e-01  -7.15332031e-01  -1.51855469e-01  -1.23107910e-01
   5.47363281e-01  -9.83886719e-02  -2.56103516e-01   5.45898438e-01
  -1.75659180e-01  -1.52709961e-01   1.86645508e-01   1.08276367e-01
   3.87954712e-03  -1.37451172e-01   1.39404297e-01   2.60009766e-02
   2.98095703e-01  -2.28393555e-01   8.85009766e-02   3.15185547e-01
  -8.91113281e-02   8.67309570e-02

In [53]:
for i,v in enumerate(vecs1):
    if v is not None:
        print text12[i], cosine(v, qv)
    else:
        print [text12[i]]

ресторан 0.610107421875
восточной 0.683837890625
кухни 0.815063476562
находится 0.769287109375
в 0.616455078125
самом 0.632568359375
центре 0.66845703125
казани 0.807983398438
посетителей 0.66357421875
ждут 0.749267578125
восхитительные 0.760986328125
блюда 0.845092773438
азербайджанской 0.8125
татарской 0.88232421875
и 0.609130859375
узбекской 0.81982421875
кухни 0.815063476562
от 0.728515625
лучших 0.544189453125
шеф 0.744873046875
поваров 0.775512695312
казани 0.807983398438
этот 0.566162109375
шикарный 0.629638671875
ресторан 0.610107421875
идеально 0.699462890625
подходит 0.697509765625
для 0.711669921875
любого 0.72802734375
повода 0.794311523438
будь 0.628173828125
то 0.55517578125
свадьба 0.689208984375
юбилей 0.611083984375
выпускной 0.615478515625
личный 0.757202148438
праздник 0.61962890625
в 0.616455078125
султанате 0.850219726562
несколько 0.732666015625
залов 0.724365234375
банкетный 0.704345703125
каминный 0.856201171875
ресторан 0.610107421875
vip 0.6279296875
зал 0.697

In [54]:
for i,v in enumerate(vecs2):
    if v is not None:
        print text22[i], cosine(v, qv)

этот 0.566162109375
бар 0.509521484375
действительно 0.55126953125
необычен 0.673828125
его 0.648193359375
создавали 0.682861328125
специально 0.664794921875
для 0.711669921875
любителей 0.539306640625
рока 0.17138671875
например 0.701171875
оформление 0.760864257812
каждого 0.75634765625
столика 0.719970703125
уникально 0.612060546875
и 0.609130859375
посвящено 0.761596679688
какому 0.71826171875
то 0.55517578125
одному 0.744140625
рок 0.0
исполнителю 0.611328125
или 0.637451171875
группе 0.54736328125
а 0.649658203125
счет 0.798950195312
здесь 0.6328125
приносят 0.769409179688
в 0.616455078125
мышеловке 0.87548828125
нет 0.6513671875
нет 0.6513671875
она 0.638427734375
не 0.61572265625
опасна 0.71484375
это 0.5556640625
просто 0.5244140625
держатель 0.772705078125
для 0.711669921875
бумаг 0.848388671875
по 0.733642578125
задумке 0.645263671875
авторов 0.619384765625
такое 0.682373046875
исполнение 0.643310546875
должно 0.739990234375
напомнить 0.765380859375
посетителям 0.734375
о 0.

In [57]:
best = sorted(cdist(np.array([v for v in vecs1 if v is not None]), [qv], 'cosine').flatten().tolist())[:5]
print best
best = sorted(cdist(np.array([v for v in vecs2 if v is not None]), [qv], 'cosine').flatten().tolist())[:5]
print best



[0.31978780496940185, 0.31978780496940185, 0.44639661776699224, 0.48286970419424113, 0.5147981617096785]
[-2.220446049250313e-16, -2.220446049250313e-16, 0.17150377607455358, 0.2117657486226805, 0.3276292392503194]


In [59]:
print 1 - np.mean([0.31978780496940185, 0.31978780496940185, 0.44639661776699224, 0.48286970419424113, 0.5147981617096785])
print 1 - np.mean([-2.220446049250313e-16, -2.220446049250313e-16, 0.17150377607455358, 0.2117657486226805, 0.3276292392503194])

0.583271981278
0.85782024721


In [13]:
scores =  [11,  2,  8,  0, 13,  3, 18, 10, 14,  1, 16,  6, 12,  7,  5, 15, 17,
        4,  9]

In [14]:
import numpy as np
scores = np.array(scores)

In [20]:
np.argsort(scores.flatten())

array([ 3,  9,  1,  5, 17, 14, 11, 13,  2, 18,  7,  0, 12,  4,  8, 15, 10,
       16,  6])